# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd
from azureml.train.automl import AutoMLConfig

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'Parkinson-classification-AutoML'

experiment=Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-132400
Azure region: southcentralus
Subscription id: cdbe0b43-92a0-4715-838a-f2648cc7ad21
Resource group: aml-quickstarts-132400


In [3]:
path_url = 'https://raw.githubusercontent.com/hananeouhammouch/Parkinsons-detection/master/parkinsons.data'
ds = TabularDatasetFactory.from_delimited_files(path = path_url)
ds.to_pandas_dataframe().to_csv("./training_dataset.csv")
datastore = ws.get_default_datastore()
datastore.upload(src_dir = "./", target_path = "data/")
training_data = TabularDatasetFactory.from_delimited_files(path = [(datastore, ("data/training_dataset.csv"))])


Uploading an estimated of 14 files
Uploading ./automl.ipynb
Uploaded ./automl.ipynb, 1 files out of an estimated total of 14
Uploading ./config.json
Uploaded ./config.json, 2 files out of an estimated total of 14
Uploading ./hyperparam_tuning.ipynb
Uploaded ./hyperparam_tuning.ipynb, 3 files out of an estimated total of 14
Uploading ./train.py
Uploaded ./train.py, 4 files out of an estimated total of 14
Uploading ./training_dataset.csv
Uploaded ./training_dataset.csv, 5 files out of an estimated total of 14
Uploading ./.ipynb_aml_checkpoints/automl-checkpoint2020-11-29-23-5-5.ipynb
Uploaded ./.ipynb_aml_checkpoints/automl-checkpoint2020-11-29-23-5-5.ipynb, 6 files out of an estimated total of 14
Uploading ./.ipynb_aml_checkpoints/hyperparam_tuning-checkpoint2020-11-29-22-56-49.ipynb
Uploaded ./.ipynb_aml_checkpoints/hyperparam_tuning-checkpoint2020-11-29-22-56-49.ipynb, 7 files out of an estimated total of 14
Uploading ./.ipynb_checkpoints/automl-checkpoint.ipynb
Uploaded ./.ipynb_chec

In [4]:
amlcompute_cluster_name = "cpu-cluster"

try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True , min_node_count = 1, timeout_in_minutes = 2)

Found existing cluster, use it.
Succeeded........................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_hours" : 0.3,
    "max_concurrent_iterations": 3,
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
    "featurization": 'auto',
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task="classification",
    compute_target=aml_compute,
    training_data=training_data,
    label_column_name="status",
    **automl_settings
)

In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
remote_run.wait_for_completion()
best_run_customized, fitted_model_customized = remote_run.get_output()


## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve and save your best automl model.
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

model_ml = best_run.register_model(model_name='Oarkinson-detection-automl', model_path='./')

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service